In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf


In [81]:
df = pd.read_csv("../Data/Combined_Delayed_2Days.csv")
df = df[["Label_delayed_2Days","Tweets_clean_more"]]
df.head()

,Label_delayed_2Days,Tweets_clean_more
0,Decrease,to super clear i dont wish could i mandat anyt...
1,Increase,ye get regulatori pushback may avail region go...
2,Decrease,dome barrel weld made bar pretti good bar need...
3,Decrease,the exec conf room tesla use call denali i dec...
4,Increase,sad hear david bowi die he amaz


In [82]:
train,test = train_test_split(df,train_size=0.8,random_state=123)
train.head()

,Label_delayed_2Days,Tweets_clean_more
1282,Decrease,rd dad empti articl one day starship land rust...
1235,Increase,exactli also definit come haha will rel recent...
597,Decrease,regard gov christi action imped direct sale au...
956,Increase,max acceler x graviti altitud later distanc ve...
550,Decrease,all system remain green launch pm est tomorrow...


In [83]:
label = preprocessing.LabelEncoder()
train["label"] = label.fit_transform(train["Label_delayed_2Days"])
test['label'] = label.transform(test['Label_delayed_2Days'])
train.head()

,Label_delayed_2Days,Tweets_clean_more,label
1282,Decrease,rd dad empti articl one day starship land rust...,0
1235,Increase,exactli also definit come haha will rel recent...,1
597,Decrease,regard gov christi action imped direct sale au...,0
956,Increase,max acceler x graviti altitud later distanc ve...,1
550,Decrease,all system remain green launch pm est tomorrow...,0


In [84]:
x_train = train['Tweets_clean_more']
x_test = test['Tweets_clean_more']

y_train = train['label']
y_test = test['label']

In [106]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [107]:
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [109]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [110]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout1")(outputs['pooled_output'])
l = tf.keras.layers.Dense(100,  activation='relu', name='output1')(l)
l = tf.keras.layers.Dropout(0.2, name="dropout2")(l)
l = tf.keras.layers.Dense(50, activation='relu', name='output2')(l)
l = tf.keras.layers.Dropout(0.2, name="dropout3")(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [111]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_3 (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [112]:
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [114]:
history = model.fit(x_train, y_train, epochs=10)

Epoch 1/10


ValueError: Creating variables on a non-first call to a function decorated with tf.function.